In [10]:
from itertools import product
import numpy as np

class Formula :

    def __init__(self) :
        pass

    def __str__(self) :
        if type(self) == Letra:
            return self.letra
        elif type(self) == Negacion:
            return '-' + str(self.subf)
        elif type(self) == Binario:
            return "(" + str(self.left) + self.conectivo + str(self.right) + ")"

    def letras(self):
        if type(self) == Letra:
            return set(self.letra)
        elif type(self) == Negacion:
            return self.subf.letras()
        elif type(self) == Binario:
            return self.left.letras().union(self.right.letras())

    def subforms(self):
        if type(self) == Letra:
            return [str(self)]
        elif type(self) == Negacion:
            return list(set([str(self)] + self.subf.subforms()))
        elif type(self) == Binario:
            return list(set([str(self)] + self.left.subforms() + self.right.subforms()))

    def valor(self, I) :
        if type(self) == Letra:
            return I[self.letra]
        elif type(self) == Negacion:
            return not self.subf.valor(I)
        elif type(self) == Binario:
            if self.conectivo == 'Y':
                return self.left.valor(I) and self.right.valor(I)
            if self.conectivo == 'O':
                return self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '>':
                return not self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '=':
                return (self.left.valor(I) and self.right.valor(I)) or (not self.left.valor(I) and not self.right.valor(I))

class Letra(Formula) :
    def __init__ (self, letra:str) :
        self.letra = letra

class Negacion(Formula) :
    def __init__(self, subf:Formula) :
        self.subf = subf

class Binario(Formula) :
    def __init__(self, conectivo:str, left:Formula, right:Formula) :
        assert(conectivo in ['Y','O','>','='])
        self.conectivo = conectivo
        self.left = left
        self.right = right

def inorder_to_tree(cadena:str):
    conectivos = ['Y', 'O', '>', '=']
    if len(cadena) == 1:
        return Letra(cadena)
    elif cadena[0] == '-':
        return Negacion(inorder_to_tree(cadena[1:]))
    elif cadena[0] == "(":
        counter = 0 #Contador de parentesis
        for i in range(1, len(cadena)):
            if cadena[i] == "(":
                counter += 1
            elif cadena[i] == ")":
                counter -=1
            elif cadena[i] in conectivos and counter == 0:
                return Binario(cadena[i], inorder_to_tree(cadena[1:i]),inorder_to_tree(cadena[i + 1:-1]))
    else:
        raise Exception('¡Cadena inválida!')

class Descriptor :

    '''
    Codifica un descriptor de N argumentos mediante un solo caracter
    Input:  args_lista, lista con el total de opciones para cada
                     argumento del descriptor
            chrInit, entero que determina el comienzo de la codificación chr()
    Output: str de longitud 1
    '''

    def __init__ (self,args_lista,chrInit=256) :
        self.args_lista = args_lista
        assert(len(args_lista) > 0), "Debe haber por lo menos un argumento"
        self.chrInit = chrInit
        self.rango = [chrInit, chrInit + np.prod(self.args_lista)]

    def check_lista_valores(self,lista_valores) :
        for i, v in enumerate(lista_valores) :
            assert(v >= 0), "Valores deben ser no negativos"
            assert(v < self.args_lista[i]), f"Valor debe ser menor o igual a {self.args_lista[i]}"

    def codifica(self,lista_valores) :
        self.check_lista_valores(lista_valores)
        cod = lista_valores[0]
        n_columnas = 1
        for i in range(0, len(lista_valores) - 1) :
            n_columnas = n_columnas * self.args_lista[i]
            cod = n_columnas * lista_valores[i+1] + cod
        return cod
    
    def escribir(self, c):
        # Implementa la lógica para convertir c en una cadena y devuélvela
        pass

    def decodifica(self,n) :
        decods = []
        if len(self.args_lista) > 1:
            for i in range(0, len(self.args_lista) - 1) :
                n_columnas = np.prod(self.args_lista[:-(i+1)])
                decods.insert(0, int(n / n_columnas))
                n = n % n_columnas
        decods.insert(0, n % self.args_lista[0])
        return decods

    def P(self,lista_valores) :
        codigo = self.codifica(lista_valores)
        return chr(self.chrInit+codigo)

    def inv(self,codigo) :
        n = ord(codigo)-self.chrInit
        return self.decodifica(n)

def visualizar_formula(A,D):
    '''
    Visualiza una fórmula A (como string en notación inorder) usando el descriptor D
    '''
    vis = []
    for c in A:
        if c == '-':
            vis.append(' no ')
        elif c in ['(', ')']:
            vis.append(c)
        elif c in ['>', 'Y', 'O']:
            vis.append(' ' + c + ' ')
        elif c == '=':
            vis.append(' sii ')
        else:
            try:
                vis.append(D.escribir(c))
            except:
                raise("¡Caracter inválido!")
    return ''.join(vis)

In [11]:
Nx = 1
Ny = 2
Objetos = ["Gallina","Zorro","Maiz"]
objeto_a_numero = {objeto: indice for indice, objeto in enumerate(Objetos)}
Nn = len(Objetos)
X = list(range(Nx))
Y = list(range(Ny))
OenP = Descriptor([Nx, Ny, Nn]) 

In [12]:
Nx = 1
Sentido = ["A a B","B a A"]
sentido_a_numero = {sent: indice for indice, sent in enumerate(Sentido)}
Objetos = ["Gallina","Zorro","Maiz"]
objeto_a_numero = {objeto: indice for indice, objeto in enumerate(Objetos)}
Nn = len(Objetos)
Ny = len(Sentido)
X = list(range(Nx))
Y = list(range(Ny))
OenS = Descriptor([Nx, Ny, Nn]) 

In [13]:
print("Cantidad de átomos OenS:", OenS.rango[1] - OenS.rango[0])
print("Caracteres correspondientes a los átomos OenS:\n")
for n in range(Nn):
    for y in range(Ny):
        for x in range(Nx):
            atomo = OenS.P([x,y,n])
            print(f"El Hombre lleva de {Sentido[y]} a ({Objetos[n]}) y es el átomo {atomo}")
    print("")

Cantidad de átomos OenS: 6
Caracteres correspondientes a los átomos OenS:

El Hombre lleva de A a B a (Gallina) y es el átomo Ā
El Hombre lleva de B a A a (Gallina) y es el átomo ā

El Hombre lleva de A a B a (Zorro) y es el átomo Ă
El Hombre lleva de B a A a (Zorro) y es el átomo ă

El Hombre lleva de A a B a (Maiz) y es el átomo Ą
El Hombre lleva de B a A a (Maiz) y es el átomo ą



In [14]:
print("Cantidad de átomos OenP:", OenP.rango[1] - OenP.rango[0])
print("Caracteres correspondientes a los átomos OenP:\n")
for n in range(Nn):
    for x in range(Nx):
        for y in range(Ny):
            atomo = OenP.P([x,y,n])
            print(f"El Hombre {Objetos[n]} está en la casilla ({X[x]},{Y[y]}) y es el átomo {atomo}")
    print("")

Cantidad de átomos OenP: 6
Caracteres correspondientes a los átomos OenP:

El Hombre Gallina está en la casilla (0,0) y es el átomo Ā
El Hombre Gallina está en la casilla (0,1) y es el átomo ā

El Hombre Zorro está en la casilla (0,0) y es el átomo Ă
El Hombre Zorro está en la casilla (0,1) y es el átomo ă

El Hombre Maiz está en la casilla (0,0) y es el átomo Ą
El Hombre Maiz está en la casilla (0,1) y es el átomo ą



In [15]:
def escribir(self, literal):
    if '-' in literal:
        atomo = literal[1:]
        neg = ' no'
    else:
        atomo = literal
        neg = ''
    x, y, n  = self.inv(atomo)
    return f" {Objetos[n]}{neg} está en la casilla ({X[x]},{Y[y]})"
    
from types import MethodType

OenP.escribir = MethodType(escribir, OenP)



atomo = OenP.P([0,1,0])
print(f"El caracter que codifica es {atomo}")
print("\nSu decodificación es:")
OenP.escribir(atomo)



El caracter que codifica es ā

Su decodificación es:


' Gallina está en la casilla (0,1)'

In [16]:
for f in Objetos:
    for y in Y:
        for x in X:

            casilla = (x,y)
            otras_casillas = [(i,j) for i in X for j in Y if (i,j) != (x,y)]
            n = objeto_a_numero[f]
            formula1 = ''
            inicial = True
            for casilla in otras_casillas:
                u = casilla[0]
                v = casilla[1]
                if inicial:
                    formula1 = OenP.P([u,v,n])
                    inicial = False
                else:
                    formula1 = "(" + formula1 + "O" + OenP.P([u,v,n]) + ")"

            formula1 = "(" + OenP.P([x,y,n]) + ">-" + formula1 + ")"
            print(visualizar_formula(formula1, OenP))
            print(" ")

( Gallina está en la casilla (0,0) >  no  Gallina está en la casilla (0,1))
 
( Gallina está en la casilla (0,1) >  no  Gallina está en la casilla (0,0))
 
( Zorro está en la casilla (0,0) >  no  Zorro está en la casilla (0,1))
 
( Zorro está en la casilla (0,1) >  no  Zorro está en la casilla (0,0))
 
( Maiz está en la casilla (0,0) >  no  Maiz está en la casilla (0,1))
 
( Maiz está en la casilla (0,1) >  no  Maiz está en la casilla (0,0))
 


In [22]:
for f in Objetos:
    for y in Y:
        for x in X:
            casilla = (x, y)
            otras_casillas = [(i, j) for i in X for j in Y if (i, j) != (x,y)]
            n = objeto_a_numero[f]
            formula1 = ''
            inicial = True
            for casilla in otras_casillas:
                u = casilla[0]
                v = casilla[1]
                if inicial:
                    formula1 = OenS.P([u, v, n])
                    inicial = False
                else:
                    formula1 = "(" + formula1 + "O" + OenS.P([u, v, n]) + ")"

            formula1 = "(" + OenS.P([x, y, n]) + ">-" + formula1 + ")"
            print(visualizar_formula(formula1, OenS))
            print(" ")

<class 'TypeError'>: sequence item 1: expected str instance, NoneType found

In [24]:
# Regla 4: La gallina y el maíz no puede quedar solos. 

for y in Y:
    for x in X:
        for n in [i for i in range(3) if i != 1]: #loop de 0 a 2 excluyendo al 1
            formula1 = ''
            inicial = True
            if inicial:
                formula1 = OenP.P([x, y, n])
                inicial = False
            else:
                formula1 = "(" + formula1 + "O" + OenP.P([x, y, n]) + ")"
            if n == 0:
                formula1 = "(" + OenP.P([x,y,2]) + ">-" + formula1 + ")"
                print(visualizar_formula(formula1, OenP))
                print(" ")
            elif n == 2:
                formula1 = "(" + OenP.P([x,y,0]) + ">-" + formula1 + ")"
                print(visualizar_formula(formula1, OenP))
                print(" ")

( Maiz está en la casilla (0,0) >  no  Gallina está en la casilla (0,0))
 
( Gallina está en la casilla (0,0) >  no  Maiz está en la casilla (0,0))
 
( Maiz está en la casilla (0,1) >  no  Gallina está en la casilla (0,1))
 
( Gallina está en la casilla (0,1) >  no  Maiz está en la casilla (0,1))
 


In [25]:
# Regla 5: La gallina y el zorro no puede quedar solos. 

for y in Y:
    for x in X:
        for n in range(2):
            formula1 = ''
            inicial = True
            if inicial:
                formula1 = OenP.P([x, y, n])
                inicial = False
            else:
                formula1 = "(" + formula1 + "O" + OenP.P([x, y, n]) + ")"
            if n == 0:
                formula1 = "(" + OenP.P([x,y,1]) + ">-" + formula1 + ")"
                print(visualizar_formula(formula1, OenP))
                print(" ")
            elif n == 1:
                formula1 = "(" + OenP.P([x,y,0]) + ">-" + formula1 + ")"
                print(visualizar_formula(formula1, OenP))
                print(" ")

( Zorro está en la casilla (0,0) >  no  Gallina está en la casilla (0,0))
 
( Gallina está en la casilla (0,0) >  no  Zorro está en la casilla (0,0))
 
( Zorro está en la casilla (0,1) >  no  Gallina está en la casilla (0,1))
 
( Gallina está en la casilla (0,1) >  no  Zorro está en la casilla (0,1))
 
